In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import numpy as np
import plotly.io as pio



In [16]:
# Set renderer to open in the browser (if not using Jupyter Notebook
pio.renderers.default = "browser"

In [17]:
# Function to parse complex values and return magnitude
def parse_complex(value):
    try:
        # Remove parentheses and convert to complex
        value = value.replace('(', '').replace(')', '')
        return abs(complex(value))  # Return magnitude
    except:
        return np.nan

In [18]:
# Paths to the data
folder_path = r"C:\Users\sasht\Downloads\archive\cleaned_dataset\data"  # Folder with impedance files
meta_file_path = r"C:\Users\sasht\Downloads\archive\cleaned_dataset\metadata.csv"  # Metadata file

In [19]:
# Step 1: Load metadata and filter for impedance type
metadata = pd.read_csv(meta_file_path)
impedance_files = metadata[metadata['type'] == 'impedance']['filename'].tolist()

In [20]:
# Step 2: Extract Battery Impedance magnitudes and combine with Re and Rct
results = []
for file in impedance_files:
    file_path = os.path.join(folder_path, file)
    try:
        # Load impedance file
        df = pd.read_csv(file_path)
        
        # Parse and calculate magnitude of Battery_impedance
        if 'Battery_impedance' in df.columns:
            df['Battery_impedance'] = df['Battery_impedance'].apply(parse_complex)
            mean_impedance = df['Battery_impedance'].mean()  # Aggregate (mean) impedance magnitude
            
            # Get Re and Rct values for the file
            re = metadata.loc[metadata['filename'] == file, 'Re'].values[0]
            rct = metadata.loc[metadata['filename'] == file, 'Rct'].values[0]
            
            # Append results with cycle number
            results.append({'Cycle': len(results) + 1, 'Re': re, 'Rct': rct, 'Battery_impedance': mean_impedance})
    except Exception as e:
        print(f"Error processing {file}: {e}")

In [ ]:
# Step 3: Create a summary DataFrame
summary_df = pd.DataFrame(results)

In [21]:
# Plot 1: Battery Impedance vs Cycle Number
fig_impedance = px.line(
    summary_df,
    x='Cycle',
    y='Battery_impedance',
    title='Battery Impedance Over Charge/Discharge Cycles',
    labels={'Cycle': 'Cycle Number', 'Battery_impedance': 'Battery Impedance (Magnitude)'}
)
fig_impedance.show()

In [22]:
# Plot 2: Re and Rct vs Cycle Number
fig_re_rct = px.line(
    summary_df.melt(id_vars='Cycle', value_vars=['Re', 'Rct']),
    x='Cycle',
    y='value',
    color='variable',
    title='Re and Rct Over Charge/Discharge Cycles',
    labels={'Cycle': 'Cycle Number', 'value': 'Resistance (Ohms)', 'variable': 'Parameter'}
)
fig_re_rct.show()

In [23]:
# Plot 3: Scatter Plot of Re vs Rct with Battery Impedance
fig_scatter = px.scatter(
    summary_df,
    x='Re',
    y='Rct',
    color='Battery_impedance',
    size='Battery_impedance',
    title='Scatter Plot of Re vs Rct with Battery Impedance',
    labels={'Re': 'Electrolyte Resistance (Ohms)', 'Rct': 'Charge Transfer Resistance (Ohms)', 'Battery_impedance': 'Impedance Magnitude'}
)
fig_scatter.show()